In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 7.8MB/s 
     |████████████████████████████████| 1.1MB 43.2MB/s 
     |████████████████████████████████| 2.9MB 47.4MB/s 
     |████████████████████████████████| 890kB 49.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a04cbca755bbdc0c29939d0f224984fe744347b90bb34fd3de895598af598a69
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm
from collections import Counter
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score
from torchtext import data
import json
import os
from torchtext import data
from collections import Counter
import torchtext
import pickle

tqdm.pandas()

In [ ]:
# !gdown 'https://drive.google.com/uc?id=1kPimR2pTbdmHGiDrczB4bW-sysvQaJ4M'
# !gdown 'https://drive.google.com/uc?id=1oACC3V2Y69DQ9A1k9x8DakU3b4cfVBHJ'
!gdown 'https://drive.google.com/uc?id=1--zGIxCA-RBK8hTxtmbc6Fwz_Vq5ONQD'

Downloading...
From: https://drive.google.com/uc?id=1--zGIxCA-RBK8hTxtmbc6Fwz_Vq5ONQD
To: /content/dict1.json
39.1MB [00:00, 183MB/s] 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = torch.load('/content/drive/MyDrive/train_data_bypublisher_bias.dataset')

val_data = torch.load('/content/drive/MyDrive/val_data_bypublisher_bias.dataset')
# byarticle_data = torch.load('/content/drive/My Drive/train_data_byarticle.dataset')
f = open('/content/dict1.json')
dict1 = json.load(f)


In [ ]:
# dict1 = json.load(f)
# train_data = torch.load()
# print(val_data)

In [ ]:
# my_dict = Counter(dict1)

In [ ]:
# vocab = torchtext.vocab.Vocab(my_dict, vectors='glove.6B.100d')

In [ ]:
batch_size = 128

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
train_on_gpu = False
if torch.cuda.is_available():
  train_on_gpu = True
print(device)
print(train_on_gpu)

cuda
True


In [ ]:
def f1_score_func(preds, labels):
    # print(preds.shape)
    # print(labels.shape)
    preds = preds.reshape(-1, 5)
    labels = labels.reshape(-1)
    preds_flat = np.argmax(preds, axis=1).flatten()
    # preds_flat = preds.flatten()
    labels_flat = labels.flatten()
    # scores_flat = scores.flatten()
#     print(preds.shape)
#     print(labels.shape)
    log_loss = nn.CrossEntropyLoss()
    
    # coef, p = spearmanr(scores_flat, preds_flat)
    return log_loss(torch.tensor(preds), torch.tensor(labels).long()), f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
# if key['bias'] == 'left':
#       key['bias'] = 0
#   elif key['bias'] == 'left-center':
#     key['bias'] = 1
#   elif key['bias'] == 'least':
#     key['bias']=2
#   elif key['bias'] == 'right-center':
#     key['bias']=3
#   else:
#     key['bias'] = 4


def accuracy_per_class(preds, labels):
    # label_dict_inverse = {v: k for k, v in label_dict.items()}
    label_dict_inverse = {4:'right',3:'right-center',2:'least',1:'left-center',0:'left'}
    preds_flat = np.argmax(preds, axis=1).flatten()
    # preds_flat = preds.flatten()
    labels_flat = labels.flatten()
    l_preds=0
    l_true=0
    accuracies = []
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        l_preds += len(y_preds[y_preds==label])
        l_true += len(y_true)
        print(f'Class: {label_dict_inverse[label]}')
        accuracies.append(f'{len(y_preds[y_preds==label])}/{len(y_true)}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)} \t {len(y_preds[y_preds==label])/len(y_true)}\n')
    print(f'Total Accuracy: {l_preds/l_true}')
    return accuracies, l_preds/l_true

In [ ]:
class SentimentRNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5, vocab=None):
        super(SentimentRNN, self).__init__()
        
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.emb_type = 'default'
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        #embedding and LSTM layers
        # copied glove vectors and training off cause pretrained
        if vocab != None:
          print('using glove')
          self.embedding.weight.data.copy_(vocab)
          self.embedding.weight.requires_grad = False
          # print('hello')
        # else:
        #   print('using glove')
        #   self.emb_type = 'glove'
        #   self.embedding = vocab
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout = drop_prob, batch_first = True)
        
        self.dropout = nn.Dropout(0.2)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        #lstm output 
        batch_size = x.size(0)
        
        x = x.long()
        # if self.emb_type == 'default':
        embeds = self.embedding(x)
        # else:
        #   embeds = self.embedding[x].reshape(x.size()[0], -1)
        #   embeds = embeds.reshape(-1, 512, 100)
        # embeds.to(device)
        # lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out, _  = self.lstm(embeds)
        #stack up lstm outputs 
        # print(lstm_out.shape)
        # lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        # print(type(lstm_out))
        out = self.dropout(lstm_out)
        out = self.fc(out[:,-1,:])
        out = self.softmax(out)
        # out = out.view(batch_size, -1, self.output_size)
        # print(out.shape)
#         print(out.shape)
        # sig_out = self.sig(out)
        
        # sig_out = sig_out.view(batch_size, -1)
        # sig_out = sig_out[:, -1] #getting the batch of labels
        #return the last sigmoid output and hidden state
        # return sig_out, hidden
        return out
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if train_on_gpu:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [ ]:
vocab_size = len(dict1)+1
output_size = 5
embedding_dim = 100
hidden_dim = 256
n_layers = 2
net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
# net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, vocab=vocab.vectors)
net.to(device)
print(net)
total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(total_params)

SentimentRNN(
  (embedding): Embedding(1995845, 100)
  (lstm): LSTM(100, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=5, bias=True)
  (sig): Sigmoid()
  (softmax): Softmax(dim=None)
)
200478713


In [ ]:
lr = 0.001
epochs = 10
counter = 0
print_every = 100
clip = 5
criterion = nn.CrossEntropyLoss()

# criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
def evaluate(val_loader):
  val_h = net.init_hidden(batch_size)
  val_losses = []
  net.eval()
  num_correct = 0
  # with torch.no_grad():
  cnt = 0
  val_preds = []
  true_vals = []
  loss_val_total = 0
  # for inputs, labels in val_loader:
  prev_out = torch.zeros(256)
  for batch in tqdm(val_loader):
      batch = tuple(b for b in batch)
      inputs, labels = batch[0], batch[1]
      if train_on_gpu:
          inputs, labels = inputs.to(device), labels.to(device)
      val_h = tuple([each.data for each in val_h])
      # net.zero_grad()
      # try:
      # output, val_h = net(inputs, val_h)
      output = net(inputs)
      # except:
      #   output = prev_out
      # print(output.shape)
      # print(labels.shape)
      val_loss = criterion(output, labels)
      loss_val_total += val_loss.item()
      prev_out = output
      # pred = torch.round(output)
      val_preds.append(output.cpu().data.numpy())
      true_vals.append(labels.float().cpu().data.numpy())
      # correct_tensor = pred.eq(labels.float().view_as(pred))
      # correct = np.squeeze(correct_tensor.cpu().numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
      # num_correct += np.sum(correct)
      # torch.cuda.empty_cache()
  # net.train()
  loss_val_avg = loss_val_total/len(val_loader)
  val_preds = np.concatenate(val_preds, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)
  # val_preds = val_preds.reshape(-1, 5)
  return loss_val_avg, val_preds, true_vals

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
# oss_val, val_preds, true_vals = evaluate(val_loader)

In [ ]:
for e in tqdm(range(1, epochs)):
    net.train()
#     # initialize hidden state
    h = net.init_hidden(batch_size)
    num_correct_train = 0
    # batch loop
    # progress_bar = tqdm(val_loader, desc='Epoch {:1d}'.format(e), leave=False, disable=False)
    progress_bar = tqdm(train_loader, desc='Epoch {:1d}'.format(e), leave=False, disable=False)
 
    for batch in progress_bar:
        counter += 1
        batch = tuple(b for b in batch)
        inputs, labels = batch[0], batch[1]
        if train_on_gpu:
            inputs, labels = inputs.to(device), labels.to(device)
 
#         # Creating new variables for the hidden state, otherwise
#         # we'd backprop through the entire training history
#         # if model_type!="GRU":
        h = tuple([each.data for each in h])
#         # else:
#         #     h = h.data
        # print(type(h))
#         # zero accumulated gradients
        net.zero_grad()
        # h.to(device)
#         # get the output from the model
        # output, h = net(inputs, h)
        output = net(inputs)
        # print(output.shape)
        # print(labels.shape)
# #         print(output.shape)
# #         print(labels.shape)
# # #         break
#         # calculate the loss and perform backprop
        loss = criterion(output, labels)
        loss.backward()
#         # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        # pred = torch.round(output)
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
 
        # correct_tensor = pred.eq(labels.float().view_as(pred))
        # correct = np.squeeze(correct_tensor.cpu().numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        # num_correct_train += np.sum(correct)
        torch.cuda.empty_cache()
    # train_acc = num_correct_train/len(train_loader.dataset)
    torch.save(net.state_dict(), f'bypublisher_lstm_{embedding_dim}_epoch_{e}.model')
    loss_val, val_preds, true_vals = evaluate(val_loader)

    _, f1 = f1_score_func(val_preds, true_vals)
    print("Epoch: {}/{}...".format(e+1, epochs),
          "Step: {}...".format(counter),
          "Loss: {:.6f}...".format(loss.item()),
          # "Train Accuracy: {:.3f}".format(train_acc),
          "Val Loss: {:.6f}".format(loss_val),
          # "Val Accuracy: {:.3f}".format(num_correct/len(val_loader.dataset)),
          "Val F1-Score: {:.3f}".format(f1))
    accuracies, total_acc = accuracy_per_class(val_preds, true_vals)
    with open('lstm_bypublisher_'+str(embedding_dim)+'_'+'glove.txt', 'a+') as out:
      out.write("Epoch: {}/{}\t".format(str(e+1), str(epochs)))
      out.write("Step: {}\t".format(str(counter)))
      out.write("Loss: {:.6f}\t".format(loss.item()))
      # out.write("Train Accuracy: {:.3f}\t".format(train_acc))
      out.write("Val Loss: {:.6f}\t".format(loss_val))
      # out.write("Val Accuracy: {:.3f}\t".format(num_correct/len(val_loader.dataset)))
      out.write("Val F1-Score: {:.3f}\n".format(f1))
      out.write(f"left: {accuracies[0]}, left-center: {accuracies[1]},least:{accuracies[2]},right-center:{accuracies[3]},right:{accuracies[4]}, Total Accuracy: {total_acc}")
    out.close()
 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



Epoch: 2/10... Step: 4687... Loss: 1.097310... Val Loss: 1.607599 Val F1-Score: 0.200
Class: left
Accuracy: 30886/37469 	 0.8243080946916117

Class: left-center
Accuracy: 9100/23456 	 0.3879604365620737

Class: least
Accuracy: 1888/38265 	 0.049340128054357765

Class: right-center
Accuracy: 3/13224 	 0.00022686025408348456

Class: right
Accuracy: 1628/37474 	 0.04344345412819555

Total Accuracy: 0.2902500533731853



Epoch: 3/10... Step: 9374... Loss: 0.993308... Val Loss: 1.624983 Val F1-Score: 0.203
Class: left
Accuracy: 26806/37474 	 0.7153226236857555

Class: left-center
Accuracy: 8883/23456 	 0.3787090723055935

Class: least
Accuracy: 1002/38261 	 0.026188547084498574

Class: right-center
Accuracy: 256/13221 	 0.019363134407382195

Class: right
Accuracy: 3738/37476 	 0.09974383605507525

Total Accuracy: 0.27143600555081127



Epoch: 4/10... Step: 14061... Loss: 1.079248... Val Loss: 1.608461 Val F1-Score: 0.233
Class: left
Accuracy: 28008/37475 	 0.747378252168112

Class: left-center
Accuracy: 6201/23453 	 0.26440114271095383

Class: least
Accuracy: 1120/38264 	 0.029270332427346852

Class: right-center
Accuracy: 145/13223 	 0.010965741511003555

Class: right
Accuracy: 8003/37473 	 0.21356710164651882

Total Accuracy: 0.29006324722459437



Epoch: 5/10... Step: 18748... Loss: 1.005490... Val Loss: 1.597786 Val F1-Score: 0.259
Class: left
Accuracy: 23421/37476 	 0.6249599743836055

Class: left-center
Accuracy: 7197/23458 	 0.30680364907494245

Class: least
Accuracy: 1320/38257 	 0.034503489557466606

Class: right-center
Accuracy: 312/13223 	 0.023595250699538682

Class: right
Accuracy: 12602/37474 	 0.33628649196776433

Total Accuracy: 0.2992367634500427



Epoch: 6/10... Step: 23435... Loss: 0.968346... Val Loss: 1.598226 Val F1-Score: 0.260
Class: left
Accuracy: 22849/37469 	 0.6098107769089114

Class: left-center
Accuracy: 6258/23452 	 0.2668429131843766

Class: least
Accuracy: 1229/38265 	 0.03211812361165556

Class: right-center
Accuracy: 301/13228 	 0.02275476262473541

Class: right
Accuracy: 14211/37474 	 0.37922292789667505

Total Accuracy: 0.2992100768573869



Epoch: 7/10... Step: 28122... Loss: 0.974517... Val Loss: 1.596815 Val F1-Score: 0.267
Class: left
Accuracy: 23247/37465 	 0.6204991325236888

Class: left-center
Accuracy: 7319/23460 	 0.3119778346121057

Class: least
Accuracy: 1687/38272 	 0.04407922240802675

Class: right-center
Accuracy: 428/13222 	 0.032370291937679624

Class: right
Accuracy: 12432/37469 	 0.3317942832741733

Total Accuracy: 0.3009780636208369



Epoch: 8/10... Step: 32809... Loss: 0.990000... Val Loss: 1.588422 Val F1-Score: 0.271
Class: left
Accuracy: 26614/37476 	 0.7101611698153485

Class: left-center
Accuracy: 5954/23458 	 0.2538153295251087

Class: least
Accuracy: 2057/38273 	 0.05374546024612651

Class: right-center
Accuracy: 240/13222 	 0.018151565572530632

Class: right
Accuracy: 11766/37459 	 0.31410341973891454

Total Accuracy: 0.31110562553373183



Epoch: 9/10... Step: 37496... Loss: 0.983667... Val Loss: 1.579828 Val F1-Score: 0.285
Class: left
Accuracy: 21994/37481 	 0.5868039806835463

Class: left-center
Accuracy: 7690/23457 	 0.3278339088544997

Class: least
Accuracy: 2280/38267 	 0.059581362531685264

Class: right-center
Accuracy: 261/13217 	 0.019747295150185366

Class: right
Accuracy: 15569/37466 	 0.41555009875620563

Total Accuracy: 0.31886475234842016



Epoch: 10/10... Step: 42183... Loss: 0.941166... Val Loss: 1.581571 Val F1-Score: 0.286
Class: left
Accuracy: 23796/37470 	 0.6350680544435549

Class: left-center
Accuracy: 7179/23456 	 0.30606241473397

Class: least
Accuracy: 2764/38262 	 0.0722387747634729

Class: right-center
Accuracy: 303/13222 	 0.022916351535319922

Class: right
Accuracy: 13507/37478 	 0.36039810021879504

Total Accuracy: 0.31723019854824935



In [ ]:
accuracies, total_acc = accuracy_per_class(val_preds, true_vals)

Class: left
Accuracy: 23796/37470 	 0.6350680544435549

Class: left-center
Accuracy: 7179/23456 	 0.30606241473397

Class: least
Accuracy: 2764/38262 	 0.0722387747634729

Class: right-center
Accuracy: 303/13222 	 0.022916351535319922

Class: right
Accuracy: 13507/37478 	 0.36039810021879504

Total Accuracy: 0.31723019854824935


In [ ]:
len(val_loader.dataset)

150000